In [312]:
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Softmax
from sklearn.model_selection import train_test_split


import numpy as np

#array de colunas

In [313]:
weeks1 = pd.read_csv('../data_raw/dados/2016-1/tabela processada nota media mediana iteracao.csv')
weeks2 = pd.read_csv('../data_raw/dados/2016-2/tabela de iteracoes com nota add.csv')
weeks3 = pd.read_csv('../data_raw/dados/2017-1/tabela final.csv')
weeks4 = pd.read_csv('../data_raw/dados/2017-2/tabela com media mediana e interacao e nota.csv')

weeks =  pd.concat([weeks1, weeks2, weeks3, weeks4])

weeks_num = 17
features = []

for i in range(weeks_num):
  features.append(f'Week{i}')
  features.append(f'MedianaW{i}')
  features.append(f'MediaW{i}')
  
features.append(f'Aprovado')


weeks.fillna(0, inplace=True)
print(features)

['Week0', 'MedianaW0', 'MediaW0', 'Week1', 'MedianaW1', 'MediaW1', 'Week2', 'MedianaW2', 'MediaW2', 'Week3', 'MedianaW3', 'MediaW3', 'Week4', 'MedianaW4', 'MediaW4', 'Week5', 'MedianaW5', 'MediaW5', 'Week6', 'MedianaW6', 'MediaW6', 'Week7', 'MedianaW7', 'MediaW7', 'Week8', 'MedianaW8', 'MediaW8', 'Week9', 'MedianaW9', 'MediaW9', 'Week10', 'MedianaW10', 'MediaW10', 'Week11', 'MedianaW11', 'MediaW11', 'Week12', 'MedianaW12', 'MediaW12', 'Week13', 'MedianaW13', 'MediaW13', 'Week14', 'MedianaW14', 'MediaW14', 'Week15', 'MedianaW15', 'MediaW15', 'Week16', 'MedianaW16', 'MediaW16', 'Aprovado']


In [314]:
#Deleting unused columns
#Converting not number values to numerical values: 1 = disapproved; 0 = approved
weeks.drop(['Unnamed: 0.1','Notas', 'Unnamed: 0', 'Nome'], inplace=True, axis=1)
weeks['Aprovado'] = pd.Categorical(weeks['Aprovado'])
weeks['Aprovado'] = weeks.Aprovado.cat.codes
weeks = weeks[features]
weeks

,Week0,MedianaW0,MediaW0,Week1,MedianaW1,MediaW1,Week2,MedianaW2,MediaW2,Week3,...,Week14,MedianaW14,MediaW14,Week15,MedianaW15,MediaW15,Week16,MedianaW16,MediaW16,Aprovado
0,15,15.0,15.000000,5,5.0,5.000000,0,0.0,0.0,0,...,0,0.0,0.000000,0,0.0,0.000000,0.0,0.0,0.0,1
1,21,10.5,10.500000,16,5.0,5.333333,3,1.5,1.5,1,...,8,2.0,2.666667,10,3.0,3.333333,5.0,2.5,2.5,1
2,3,3.0,3.000000,20,9.0,6.666667,1,1.0,1.0,0,...,0,0.0,0.000000,0,0.0,0.000000,2.0,2.0,2.0,1
3,350,9.0,116.666667,1568,297.0,313.600000,1054,284.0,263.5,13,...,124,11.0,41.333333,175,87.5,87.500000,1.0,1.0,1.0,0
4,6,6.0,6.000000,11,2.5,2.750000,5,2.5,2.5,1,...,12,6.0,6.000000,4,1.0,1.333333,4.0,2.0,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,0,0.0,0.000000,0,0.0,0.000000,2,2.0,2.0,0,...,0,0.0,0.000000,0,0.0,0.000000,0.0,0.0,0.0,1
28,0,0.0,0.000000,0,0.0,0.000000,0,0.0,0.0,0,...,3,3.0,3.000000,34,17.0,17.000000,0.0,0.0,0.0,0
29,0,0.0,0.000000,0,0.0,0.000000,0,0.0,0.0,0,...,0,0.0,0.000000,0,0.0,0.000000,0.0,0.0,0.0,1
30,0,0.0,0.000000,0,0.0,0.000000,0,0.0,0.0,0,...,0,0.0,0.000000,0,0.0,0.000000,0.0,0.0,0.0,1


In [316]:
weeks = weeks.to_numpy()
np.random.shuffle(weeks)

X, y = np.split(weeks,[-1],axis=1) # Or simply : np.split(Xy,[-1],1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)

X_train.shape

(73, 51)

In [317]:
model = Sequential()
model.add(LSTM(10, activation='relu', input_shape=((weeks_num*3), 1)))
model.add(Dense(1))
model.add(Softmax(1))
model.compile(optimizer='adam', loss='mse')

In [318]:
history = model.fit(X_train, y_train, epochs=200, verbose=2)

Epoch 1/200
3/3 - 3s - loss: 0.3425 - 3s/epoch - 1s/step
Epoch 2/200
3/3 - 0s - loss: 0.3425 - 131ms/epoch - 44ms/step
Epoch 3/200
3/3 - 0s - loss: 0.3425 - 117ms/epoch - 39ms/step
Epoch 4/200
3/3 - 0s - loss: 0.3425 - 136ms/epoch - 45ms/step
Epoch 5/200
3/3 - 0s - loss: 0.3425 - 120ms/epoch - 40ms/step
Epoch 6/200
3/3 - 0s - loss: 0.3425 - 94ms/epoch - 31ms/step
Epoch 7/200
3/3 - 0s - loss: 0.3425 - 111ms/epoch - 37ms/step
Epoch 8/200
3/3 - 0s - loss: 0.3425 - 113ms/epoch - 38ms/step
Epoch 9/200
3/3 - 0s - loss: 0.3425 - 117ms/epoch - 39ms/step
Epoch 10/200
3/3 - 0s - loss: 0.3425 - 129ms/epoch - 43ms/step
Epoch 11/200
3/3 - 0s - loss: 0.3425 - 124ms/epoch - 41ms/step
Epoch 12/200
3/3 - 0s - loss: 0.3425 - 131ms/epoch - 44ms/step
Epoch 13/200
3/3 - 0s - loss: 0.3425 - 134ms/epoch - 45ms/step
Epoch 14/200
3/3 - 0s - loss: 0.3425 - 123ms/epoch - 41ms/step
Epoch 15/200
3/3 - 0s - loss: 0.3425 - 134ms/epoch - 45ms/step
Epoch 16/200
3/3 - 0s - loss: 0.3425 - 112ms/epoch - 37ms/step
Epoch 1

In [319]:
acc = model.evaluate(X_test, y_test, batch_size =54)
print(1-acc)

1/1 [==============================] - 0s 384ms/step - loss: 0.2083
0.7916666716337204


In [320]:
score_val = model.evaluate(X_val, y_val)
print(1-score_val)

1/1 [==============================] - 0s 47ms/step - loss: 0.2000
0.7999999970197678
